In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
import pandas as pd
from time import sleep
from random import randint
import datetime

In [2]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
city = 'Los Angeles'

# Create empty list to store scraped data
datatoget = []

for i in range(0,8):    
    
    # Set url
    url = f'https://sfbay.craigslist.org/search/apa#search=1~thumb~{i}~0'
    browser.visit(url)
    sleep(randint(5,7))

    # Parse the HTML
    html = browser.html
    soup = bs(html, 'html.parser')

    # Find and extract relevant data
    search = soup.find_all('div', class_='result-info')

    print('\n================')
    print(f'This is page {i+1}')
    print('================')

    # Loop through data found and extract relevant information
    for (counter,link) in enumerate(search):
            
        # Scrape dates
        try:
            dates = link.find_all('div', class_='meta')[0].text.split('·')[0]

            current_date = datetime.datetime.now()

            if link.find_all('div', class_='meta')[0].text.split('·')[0][-9:] == 'hours ago':
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')

            elif link.find_all('div', class_='meta')[0].text.split('·')[0][-11:] == 'minutes ago':
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')
            
            elif link.find_all('div', class_='meta')[0].text.split('·')[0][-8:] == 'hour ago':
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')
            
            else:
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')

        except:
            dates = np.nan

        # Scrape titles
        titles = link.find_all('a', class_='titlestring')[0].text

        # Scrape links
        links = link.find_all('a', class_='titlestring', href=True)[0]['href']

        try:
        # Scrape prices
            prices = link.find_all('span', class_='priceinfo')[0].text
        except:
            prices = np.nan

        # Scrape neighborhoods
        try:
            neighborhoods = link.find('div', class_='supertitle').text
        
        except:
            neighborhoods = np.nan

        # Visit each link seperately
        browser.visit(links)

        # Ensures that the server does not get overloaded with requests
        sleep(randint(3,5))  

        # After visiting link, extract bath and amenities data
        html = browser.html
        soup = bs(html, 'html.parser')

        # Error handeling in case there are no sqft displayed
        try:
            if soup.find_all('p', class_='attrgroup')[0].text.split('\n')[2][-3:] == 'ft2':
                sqft = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[2][:-3]
            else:
                sqft = np.nan
        except:
            sqft = np.nan
        
        # Error handeling in case there are no bedrooms displayed
        try:
            bedrooms = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[1].split('/')[0].strip().lower()

        except:
            bedrooms = np.nan
        
        try:
            baths = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[1].split()[2][:-2]

        except:
            baths = np.nan

        try:
            amenities = soup.find_all('p', class_='attrgroup')[1].text.split('\n')
            # List comprehension used to make sure no empty strings get added to the list
            all_amenities = [am for am in amenities if am != '']
        
        except:
              all_amenities = np.nan
              
        # Print statements to ensure that the correct data is scraped
        print(f'\nListing Number: {counter+1}')
        print(f'Square Footage: {sqft if sqft == sqft else "not available"}')
        print(f'Price: {prices if prices == prices else "not available"}')
        print(f'Title: {titles}')
        print(f'Date: {dates}')
        print(f'Number of Bedrooms: {bedrooms if bedrooms == bedrooms else "not available"}')
        print(f'Neighborhood: {neighborhoods if neighborhoods == neighborhoods else "not available"}')
        print(f'Number of Bathrooms: {baths if baths == baths else "not available"}')
        print(f'Link: {links}')
        print(f'Amenities: {all_amenities if all_amenities == all_amenities else "not available"}')

        # Create dicitonary for our DataFrame
        data = {'date': dates,
                'title': titles,
                'link': links,
                'price': prices,
                'bedroom': bedrooms,
                'sqft': sqft,
                'neighborhood': neighborhoods,
                'bathroom': baths,
                'amenities': all_amenities}

        # Append the data
        datatoget.append(data)

print('\n================ Done ================')

# Quit automated browser instance
browser.quit()


This is page 1

Listing Number: 1
Square Footage: 350
Price: $1,350
Title: NICE SINGLE ON RESIDENTIAL STREET
Date: 1/17
Number of Bedrooms: 0br
Neighborhood: 2946 W. 14TH STREET
Number of Bathrooms: 1
Link: https://losangeles.craigslist.org/lac/apa/d/los-angeles-nice-single-on-residential/7570850785.html
Amenities: ['application fee details: 30.00', 'apartment', 'no laundry on site', 'street parking', 'rent period: monthly']

Listing Number: 2
Square Footage: 550
Price: $1,350
Title: NICE SINGLE ON RESIDENTIAL STREET
Date: 1/17
Number of Bedrooms: 0br
Neighborhood: 2946 W. 14TH STREET
Number of Bathrooms: 1
Link: https://losangeles.craigslist.org/lac/apa/d/los-angeles-nice-single-on-residential/7573546051.html
Amenities: ['application fee details: 30.00', 'apartment', 'no laundry on site', 'no smoking', 'street parking', 'rent period: monthly']

Listing Number: 3
Square Footage: 700
Price: $2,195
Title: Covered Parking, Stainless Electric Stove, Recycling
Date: 1/17
Number of Bedrooms


Listing Number: 20
Square Footage: 1200
Price: $3,095
Title: La Buena Ventura Apts.Brand New Building.3 Bedroom 2 Bath-Amazing View
Date: 1/17
Number of Bedrooms: 3br
Neighborhood: Panorama City
Number of Bathrooms: 2
Link: https://losangeles.craigslist.org/sfv/apa/d/panorama-city-la-buena-ventura/7576159262.html
Amenities: ['application fee details: $45', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry on site', 'carport', 'rent period: monthly']

Listing Number: 21
Square Footage: 1010
Price: $3,880
Title: Walk-in Closets with Custom Storage, Samsung Washer & Dryer
Date: 1/17
Number of Bedrooms: 2br
Neighborhood: North Hollywood
Number of Bathrooms: 2
Link: https://losangeles.craigslist.org/wst/apa/d/north-hollywood-walk-in-closets-with/7579725218.html
Amenities: ['EV charging', 'air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'attached garage', 'rent period: monthly', 'wheelchair accessible']

Listing Nu


Listing Number: 39
Square Footage: 900
Price: $3,718
Title: One block to UCLA, On-Site Management, On-Site Maintenance
Date: 1/17
Number of Bedrooms: 2br
Neighborhood: 415 Gayley Ave, Los Angeles, CA
Number of Bathrooms: 1
Link: https://losangeles.craigslist.org/wst/apa/d/los-angeles-one-block-to-ucla-on-site/7574884042.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry on site', 'off-street parking', 'rent period: monthly']

Listing Number: 40
Square Footage: 844
Price: $1,635
Title: Microwave, BBQ/Picnic Area, Patio/Balcony
Date: 1/17
Number of Bedrooms: 2br
Neighborhood: 43230 Gadsden Avenue, Lancaster, CA
Number of Bathrooms: 1
Link: https://losangeles.craigslist.org/ant/apa/d/lancaster-microwave-bbq-picnic-area/7576965023.html
Amenities: ['apartment', 'laundry on site', 'carport', 'rent period: monthly']

Listing Number: 41
Square Footage: 400
Price: $1,450
Title: Irolo Apts..New York Loft Style Gorgeous Studio...Dog Friendly!
Date: 1/17
Number o


Listing Number: 57
Square Footage: 573
Price: $1,850
Title: $500 off! Microwave, Walking Distance to CSUN
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: 18520 Vincennes St, Northridge, CA
Number of Bathrooms: 1
Link: https://losangeles.craigslist.org/sfv/apa/d/northridge-500-off-microwave-walking/7576700178.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry on site', 'off-street parking', 'rent period: daily']

Listing Number: 58
Square Footage: 400
Price: $1,550
Title: Lexington Ave Apts...Newly Remodeled Gorgeous Studio...Great Location!
Date: 1/17
Number of Bedrooms: 0br
Neighborhood: Hollywood
Number of Bathrooms: 1
Link: https://losangeles.craigslist.org/lac/apa/d/los-angeles-lexington-ave-aptsnewly/7572582173.html
Amenities: ['application fee details: $45', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry on site', 'street parking', 'rent period: monthly']

Listing Number: 59
Square Footage: 984
Price: $3,309
Title: Crown Mo


Listing Number: 76
Square Footage: 800
Price: $2,200
Title: Wood Style Floors + A/C + Quartz Counters + Gym + 1 Bedroom in Tarzana
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: Tarzana
Number of Bathrooms: 1
Link: https://losangeles.craigslist.org/sfv/apa/d/tarzana-wood-style-floors-c-quartz/7579708672.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'apartment', 'laundry on site', 'off-street parking', 'rent period: monthly']

Listing Number: 77
Square Footage: not available
Price: $1,925
Title: Great updated units in Encino, Close to shops and stores
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: ENCINO
Number of Bathrooms: 1
Link: https://losangeles.craigslist.org/sfv/apa/d/great-updated-units-in-encino-close-to/7576602486.html
Amenities: ['open house dates', 'wednesday 2023-01-18', 'thursday 2023-01-19', 'friday 2023-01-20']

Listing Number: 78
Square Footage: 774
Price: $2,520
Title: 1 bedroom, 18-foot ceilings
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: long

...


Listing Number: 34
Square Footage: 585
Price: $2,308
Title: 24-Hour fitness center, Ceiling fans, Private patio or balcony
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: 5351 San Vicente Boulevard, Los Angeles, CA
Number of Bathrooms: 1
Link: https://losangeles.craigslist.org/lac/apa/d/los-angeles-24-hour-fitness-center/7578915338.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry on site', 'detached garage', 'rent period: monthly']

Listing Number: 35
Square Footage: 860
Price: $2,121
Title: Picnic Tables, Expansive Sunbathing Areas, Children's Play Area
Date: 1/17
Number of Bedrooms: 2br
Neighborhood: 27303 N. Sara St., Los Angeles, CA
Number of Bathrooms: 2
Link: https://losangeles.craigslist.org/sgv/apa/d/canyon-country-picnic-tables-expansive/7576073608.html
Amenities: ['apartment', 'w/d in unit', 'carport', 'rent period: monthly']

Listing Number: 36
Square Footage: 775
Price: $2,895
Title: Prime Mar Vista 2BD 1BA First Floor New Kitchens AC, 


Listing Number: 53
Square Footage: 950
Price: $1,660
Title: Dos Cuartos y un Bano / Two Bed_One bath
Date: 1/17
Number of Bedrooms: 2br
Neighborhood: central LA 213/323
Number of Bathrooms: 1
Link: https://losangeles.craigslist.org/lac/apa/d/paramount-dos-cuartos-un-bano-two/7579662361.html
Amenities: ['apartment', 'laundry in bldg', 'detached garage', 'rent period: monthly']

Listing Number: 54
Square Footage: not available
Price: $2,195
Title: 1 Bedroom 1 Bath In Encino + Pool, Gym, Spa, CALL US Today!
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: Encino
Number of Bathrooms: 1
Link: https://losangeles.craigslist.org/sfv/apa/d/encino-bedroom-bath-in-encino-pool-gym/7579673810.html
Amenities: ['apartment', 'laundry on site', 'off-street parking', 'rent period: monthly']

Listing Number: 55
Square Footage: 1900
Price: $4,995
Title: HOUSE WITH POOL, WOOD FLOOR, CENTRAL AC, GRANITE, WASHER DRYER, PET OK
Date: 1/17
Number of Bedrooms: 4br
Neighborhood: ************** NORTH HOLLYWOOD **


Listing Number: 72
Square Footage: 450
Price: $1,810
Title: State-Of-The-Art Fitness Center, Air Conditioning*
Date: 1/17
Number of Bedrooms: 0br
Neighborhood: Lakewood, Bixby Knolls, Signal Hill
Number of Bathrooms: 1
Link: https://losangeles.craigslist.org/lgb/apa/d/lakewood-state-of-the-art-fitness/7579672615.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry on site', 'carport', 'rent period: monthly']

Listing Number: 73
Square Footage: 470
Price: $1,955
Title: Convenient location in Studio City - Come see our 1 Bed / 1 bath!
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: Studio City - Sherman Oaks
Number of Bathrooms: 1
Link: https://losangeles.craigslist.org/sfv/apa/d/sherman-oaks-convenient-location-in/7579672559.html
Amenities: ['apartment', 'laundry on site', 'attached garage', 'rent period: monthly']

Listing Number: 74
Square Footage: 1100
Price: $2,995
Title: Relaxing Jacuzzi, Assigned Uncovered Parking, Carpeting
Da


Listing Number: 91
Square Footage: not available
Price: $2,750
Title: *********PRIME GLENDALE LOCATION, FULLY RENOVATED HOME ***************
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: ************** GLENDALE ***************
Number of Bathrooms: 1
Link: https://losangeles.craigslist.org/sfv/apa/d/glendale-prime-glendale-location-fully/7579671805.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'duplex', 'w/d in unit', 'off-street parking', 'rent period: monthly']

Listing Number: 92
Square Footage: not available
Price: $2,595
Title: Westwood 1 Bed 1 Bath w/ Full Kitchen, Parking Near UCLA
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: Westwood/90024/UCLA
Number of Bathrooms: 1
Link: https://losangeles.craigslist.org/wst/apa/d/los-angeles-westwood-bed-bath-full/7577489051.html
Amenities: ['air conditioning', 'apartment', 'laundry on site', 'attached garage', 'rent period: monthly', 'wheelchair accessible']

Listing Number: 93
Square Footage: 470
Price: $1,955
Title


Listing Number: 109
Square Footage: 457
Price: $2,450
Title: Outdoor Fitness Center, Dogs and Cats Welcome!, Library
Date: 1/17
Number of Bedrooms: 0br
Neighborhood: 4600 Vía Marina, Marina del Rey, CA
Number of Bathrooms: 1
Link: https://losangeles.craigslist.org/wst/apa/d/marina-del-rey-outdoor-fitness-center/7578896794.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry on site', 'carport', 'rent period: monthly']

Listing Number: 110
Square Footage: 819
Price: $2,185
Title: Incredible space. Inviting location. 1 bd 1 ba. Make a home with us!
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: Los Angeles
Number of Bathrooms: 1
Link: https://losangeles.craigslist.org/lac/apa/d/los-angeles-incredible-space-inviting/7579669192.html
Amenities: ['EV charging', 'air conditioning', 'application fee details: 53.00', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry on site', 'attached garage', 'rent period: monthly', 'wheelchair accessible']

In [4]:
# creating DataFrame
df = pd.DataFrame(datatoget)
df

,date,title,link,price,bedroom,sqft,neighborhood,bathroom,amenities
0,1/17,NICE SINGLE ON RESIDENTIAL STREET,https://losangeles.craigslist.org/lac/apa/d/lo...,"$1,350",0br,350,2946 W. 14TH STREET,1,"[application fee details: 30.00, apartment, no..."
1,1/17,NICE SINGLE ON RESIDENTIAL STREET,https://losangeles.craigslist.org/lac/apa/d/lo...,"$1,350",0br,550,2946 W. 14TH STREET,1,"[application fee details: 30.00, apartment, no..."
2,1/17,"Covered Parking, Stainless Electric Stove, Rec...",https://losangeles.craigslist.org/lac/apa/d/lo...,"$2,195",1br,700,"1738 N. Canyon Dr, Los Angeles, CA",1,"[air conditioning, cats are OK - purrr, dogs a..."
3,1/17,"Closet Organizers, Soothing Jacuzzi Spa, Pantry",https://losangeles.craigslist.org/lgb/apa/d/lo...,"$2,485",1br,646,"909 W. Temple St. , Los Angeles, CA",1,"[open house dates, monday 2023-01-09, tuesday ..."
4,1/17,Very nice Single with full kitchen and bathroom,https://losangeles.craigslist.org/lac/apa/d/ve...,"$1,350",0br,500,"1039 W. 58th Street, Los Angeles",1,"[application fee details: 30.00, apartment, no..."
...,...,...,...,...,...,...,...,...,...
955,1/17,You've never seen anything like these deals! C...,https://losangeles.craigslist.org/lac/apa/d/lo...,"$1,875",1br,NaN,Los Angeles - Melrose Hill,1,"[air conditioning, application fee details: $5..."
956,1/17,"1 Bed 1 Bath in Prime Van Nuys | A/C, Refriger...",https://losangeles.craigslist.org/sfv/apa/d/pl...,"$1,850",1br,675,Van Nuys 91405,1,"[apartment, laundry on site, off-street parkin..."
957,1/17,BBQ/Picnic Area | Sauna | Gym | 1 Bedroom in C...,https://losangeles.craigslist.org/wst/apa/d/lo...,"$2,400",1br,700,OPEN HOUSE DAILY 10-5 | West Los Angeles,1,"[EV charging, air conditioning, cats are OK - ..."
958,1/17,"24-Hour Doorman, Pantry, Nine Foot Ceilings",https://losangeles.craigslist.org/sfv/apa/d/lo...,"$2,762",2br,858,"909 W. Temple St., Los Angeles, CA",2,"[apartment, w/d in unit, carport, rent period:..."


In [5]:
# Exporting data to csv
df.to_csv(f"{city.replace(' ', '_')}_data_raw.csv", index=False)